In [ ]:
! pip install -q kaggle
! mkdir ~/kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/kaggle’: File exists


In [ ]:
from google.colab import drive
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!kaggle datasets download mohamedmustafa/real-life-violence-situations-dataset -p /content/rlvs/ --unzip

100% 3.58G/3.58G [02:39<00:00, 24.6MB/s]
100% 3.58G/3.58G [02:39<00:00, 24.1MB/s]


In [ ]:
!kaggle datasets download yassershrief/hockey-fight-vidoes -p /content/hockey --unzip

 98% 161M/164M [00:07<00:00, 25.1MB/s]
100% 164M/164M [00:07<00:00, 21.6MB/s]


In [ ]:
!unzip '/content/drive/MyDrive/Capstone Resources/Phase 1/Videos/Non-violent/non-violence.zip' -d '/content/non-violence/'
!unzip '/content/drive/MyDrive/Capstone Resources/Phase 1/Videos/violent.zip' -d '/content/violence/'

Archive:  /content/drive/MyDrive/Capstone Resources/Phase 1/Videos/Non-violent/non-violence.zip
  inflating: /content/non-violence/chores1.gif  
  inflating: /content/non-violence/chores10.gif  
  inflating: /content/non-violence/chores2.gif  
  inflating: /content/non-violence/chores3.gif  
  inflating: /content/non-violence/chores4.gif  
  inflating: /content/non-violence/chores5.gif  
  inflating: /content/non-violence/chores6.gif  
  inflating: /content/non-violence/chores7.gif  
  inflating: /content/non-violence/chores8.gif  
  inflating: /content/non-violence/chores9.gif  
  inflating: /content/non-violence/cooking1.gif  
  inflating: /content/non-violence/cooking2.gif  
  inflating: /content/non-violence/cooking3.gif  
  inflating: /content/non-violence/cooking4.gif  
  inflating: /content/non-violence/dancing1.gif  
  inflating: /content/non-violence/dancing10.gif  
  inflating: /content/non-violence/dancing2.gif  
  inflating: /content/non-violence/dancing3.gif  
  inflating:

In [ ]:
import os
import cv2
import tensorflow as tf
import numpy as np

In [ ]:
!unzip /content/model_88.zip -d /content/model_88

Archive:  /content/model_88.zip
   creating: /content/model_88/content/model_88/
  inflating: /content/model_88/content/model_88/keras_metadata.pb  
   creating: /content/model_88/content/model_88/variables/
  inflating: /content/model_88/content/model_88/variables/variables.data-00000-of-00001  
  inflating: /content/model_88/content/model_88/variables/variables.index  
   creating: /content/model_88/content/model_88/assets/
  inflating: /content/model_88/content/model_88/saved_model.pb  


In [ ]:
model=tf.keras.models.load_model('/content/model_88/content/model_88')

In [ ]:
MAX_SEQ_LENGTH = 30
IMG_SIZE = 90

def resize_frames(frame):
    resize_layer = tf.keras.layers.Resizing(
        IMG_SIZE, IMG_SIZE, crop_to_aspect_ratio=True)
    #resize_layer=tf.keras.layers.CenterCrop(IMG_SIZE,IMG_SIZE)
    resized = resize_layer(frame[None, ...])
    resized = resized.numpy().squeeze()
    return resized

def load_video(path, max_frames):
    frames_to_take_per_sec=3
    cap = cv2.VideoCapture(path)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    frames = []
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    k=int(fps/frames_to_take_per_sec)
    i=0
    j=0
    try:
      while j<max_frames:
        ret, frame = cap.read()
        if not ret:
          break
        if i%k==0:
          j+=1
          frame= resize_frames(frame)
          frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
          frames.append(frame)
        i+=1
    finally:
      cap.release()
      return np.array(frames[:max_frames])


def prepare_single_video(frames):
    if len(frames) < MAX_SEQ_LENGTH:
            diff = MAX_SEQ_LENGTH - len(frames)
            padding = np.empty((diff, IMG_SIZE, IMG_SIZE), dtype="float32")
            for d in range(diff):
              padding[d:]=frames[-1:]
            frames = np.concatenate((frames, padding))


    return frames


def predict_action(path):
    class_vocab = ['non-violent', 'violent']
    frames = load_video(path,MAX_SEQ_LENGTH)
    video = prepare_single_video(frames)
    pred = model.predict(tf.expand_dims(video, axis=0))[0]
    if pred>0.5:
      return class_vocab[1]
    else:
      return class_vocab[0]


In [ ]:


rlvs_nv = "/content/rlvs/Real Life Violence Dataset/NonViolence"
v=0
for vid in os.listdir(rlvs_nv):
  print(os.path.join(rlvs_nv,vid))
  if predict_action(os.path.join(rlvs_nv,vid))=="non-violent":
    v+=1
print("Accuracy on rlvs non violence videos",round(v*100/len(os.listdir(rlvs_nv)),2),"%")


/content/rlvs/Real Life Violence Dataset/NonViolence/NV_825.mp4
1/1 [==============================] - 7s 7s/step
/content/rlvs/Real Life Violence Dataset/NonViolence/NV_223.mp4
1/1 [==============================] - 0s 23ms/step
/content/rlvs/Real Life Violence Dataset/NonViolence/NV_121.mp4
1/1 [==============================] - 0s 21ms/step
/content/rlvs/Real Life Violence Dataset/NonViolence/NV_744.mp4
1/1 [==============================] - 0s 26ms/step
/content/rlvs/Real Life Violence Dataset/NonViolence/NV_887.avi
1/1 [==============================] - 0s 24ms/step
/content/rlvs/Real Life Violence Dataset/NonViolence/NV_42.mp4
1/1 [==============================] - 0s 21ms/step
/content/rlvs/Real Life Violence Dataset/NonViolence/NV_775.mp4
1/1 [==============================] - 0s 25ms/step
/content/rlvs/Real Life Violence Dataset/NonViolence/NV_847.mp4
1/1 [==============================] - 0s 22ms/step
/content/rlvs/Real Life Violence Dataset/NonViolence/NV_968.mp4
1/1 [======

In [ ]:

rlvs_v = "/content/rlvs/Real Life Violence Dataset/Violence"
v=0
for vid in os.listdir(rlvs_v):
  if predict_action(os.path.join(rlvs_v,vid))=="violent":
    v+=1
print("Accuracy on rlvs violence videos",round(v*100/len(os.listdir(rlvs_v)),2),"%")

1/1 [==============================] - 0s 23ms/step
Accuracy on rlvs violence videos 97.6 %


In [ ]:
data=os.listdir("/content/hockey/data")

data = sorted(data)
violent = data[:500]
non_violent = data[500:]

v=0
for vid in non_violent:
  if predict_action(os.path.join("/content/hockey/data",vid))=="non-violent":
    v+=1
print("Accuracy on hockey non violence videos",round(v*100/500,2),"%")



1/1 [==============================] - 0s 24ms/step
Accuracy on hockey non violence videos 66.2 %


In [ ]:
v=0
for vid in violent:
  if predict_action(os.path.join("/content/hockey/data",vid))=="violent":
    v+=1
print("Accuracy on hockey violence videos",round(v*100/500,2),"%")

1/1 [==============================] - 0s 23ms/step
Accuracy on hockey violence videos 51.2 %


In [ ]:


custom_nv ="/content/non-violence"
v=0
for vid in os.listdir(custom_nv):
  if predict_action(os.path.join(custom_nv,vid))=="non-violent":
    v+=1
print("Accuracy on custom non violence videos",round(v*100/len(os.listdir(custom_nv)),2),"%")



1/1 [==============================] - 0s 22ms/step
Accuracy on custom non violence videos 89.55 %


In [ ]:
custom_v = "/content/violence"
v=0
for vid in os.listdir(custom_v):
  if predict_action(os.path.join(custom_v,vid))=="violent":
    v+=1
print("Accuracy on custom violence videos",round(v*100/len(os.listdir(custom_v)),2),"%")

1/1 [==============================] - 0s 26ms/step
Accuracy on custom violence videos 98.92 %
